In [1]:
# 1. Install necessary libraries
# We use 'quiet' to keep the output clean, remove it if you want to see installation details
!pip install pandas nltk textblob --quiet

# 2. Import modules
import pandas as pd
import random
import json
from datetime import datetime
import nltk
from textblob import TextBlob # Good for simple sentiment detection (keeping mood positive)

# 3. Download required NLTK data for tokenizing text
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True) # Useful corpus for reference

# 4. Define a simple configuration for the bot
BOT_CONFIG = {
    "bot_name": "Remi", # Short for Reminiscence
    "user_name": "User", # We can change this dynamically later
    "session_start": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

print(f"✅ Environment Ready for {BOT_CONFIG['bot_name']}!")
print(f"Session started at: {BOT_CONFIG['session_start']}")

✅ Environment Ready for Remi!
Session started at: 2026-02-08 22:35:44


From the code above, 

pandas: We will use this to save conversation logs (very important for analysis later).

textblob: A simple library to detect sentiment. If a user gets agitated (negative sentiment), we can program the bot to change the subject.

BOT_CONFIG: A dictionary to store basic settings we can reference later.

We now want to move to Step 2: Defining the "Reminiscence Knowledge Base" This is the brain the bot will draw from.

In [2]:
# Define the topics and specific reminiscence prompts
reminiscence_topics = {
    "childhood": [
        "What was your favorite game to play outside when you were young?",
        "Do you remember the house you grew up in? What was your favorite room?",
        "What was your favorite home-cooked meal as a child?",
        "Did you have any pets growing up? What were their names?"
    ],
    "career": [
        "What was your very first job? Did you enjoy it?",
        "What is a professional accomplishment you are most proud of?",
        "Who was the best boss or mentor you ever had?",
        "How did your work change over the years?"
    ],
    "hobbies": [
        "What hobbies have you enjoyed throughout your life?",
        "Is there a particular song or type of music that always makes you smile?",
        "Did you enjoy traveling? Where was your favorite place to visit?",
        "What is a book or movie that had a big impact on you?"
    ],
    "family": [
        "How did you meet your spouse or partner?",
        "What is a favorite holiday tradition your family had?",
        "Tell me about a memorable family vacation.",
        "What values were most important to your parents?"
    ]
}

# Simple function to get a random question from a topic
def get_prompt(topic):
    if topic in reminiscence_topics:
        return random.choice(reminiscence_topics[topic])
    else:
        return "I'd love to hear more about that. Can you tell me more?"

print("✅ Knowledge Base Loaded with topics:", list(reminiscence_topics.keys()))

✅ Knowledge Base Loaded with topics: ['childhood', 'career', 'hobbies', 'family']


We are keeping the conversation focused on therapeutic areas and later we can add other information like music, school, e.t.c.

Defining The Logic Functions:

Next, we are going to build the "Brain" It is a function that takes what the user says, analyzes it and decides the memory topic to trigger next.

In [5]:
# --- REMI V3: The "Context Aware" Update ---

# 1. We keep track of used questions here so we never repeat them
chat_state = {
    "current_topic": None,
    "asked_questions": set() # Using a set for faster lookups
}

def get_new_prompt(topic):
    """
    Picks a question we haven't asked yet.
    If we've asked them all, it returns None.
    """
    available_questions = reminiscence_topics.get(topic, [])
    
    # Filter out questions we already asked
    fresh_questions = [q for q in available_questions if q not in chat_state["asked_questions"]]
    
    if fresh_questions:
        selected = random.choice(fresh_questions)
        chat_state["asked_questions"].add(selected) # Mark as used
        return selected
    else:
        return None # No questions left in this topic!

def is_negative_answer(text):
    """Checks if the user said 'no', 'none', 'I don't', etc."""
    triggers = ["no", "nope", "i don't", "i do not", "nothing", "none", "never", "did not"]
    return any(trigger in text.lower() for trigger in triggers)

def get_smart_response_v3(user_input):
    # 1. Check for immediate hostility/distress
    mood = analyze_sentiment(user_input)
    if mood < -0.3 or "stop" in user_input.lower() or "wrong with you" in user_input.lower():
        chat_state["current_topic"] = "hobbies"
        return "I apologize. I am just a computer program trying to learn. Let's start over. do you have a favorite animal?"

    # 2. Check for 'No' answers (The "Drop It" Logic)
    if is_negative_answer(user_input):
        # If user says "No" or "I don't", immediately switch topics
        old_topic = chat_state["current_topic"]
        available_topics = list(reminiscence_topics.keys())
        
        # Pick a topic that isn't the current one
        if old_topic in available_topics:
            available_topics.remove(old_topic)
            
        new_topic = random.choice(available_topics)
        chat_state["current_topic"] = new_topic
        
        question = get_new_prompt(new_topic)
        return f"Understood. Let's move on. {question}"

    # 3. Handle very short/nonsense answers (The "Clarification" Logic)
    if len(user_input) < 4 and not user_input.lower() in ['yes', 'ok']:
        return "I'm not sure I understand. Could you tell me a bit more in a full sentence?"

    # 4. Standard Topic Detection
    detected = detect_topic(user_input)
    if detected:
        chat_state["current_topic"] = detected
        question = get_new_prompt(detected)
        return f"Ah, {detected}. {question}"
    
    # 5. Continue Current Conversation
    if chat_state["current_topic"]:
        question = get_new_prompt(chat_state["current_topic"])
        
        # If we ran out of questions for this topic, force a switch
        if not question:
            new_topic = random.choice(list(reminiscence_topics.keys()))
            chat_state["current_topic"] = new_topic
            question = get_new_prompt(new_topic)
            return f"We've talked a lot about that. Let's switch to {new_topic}. {question}"
            
        return f"{random.choice(acknowledgments)} {question}"

    # 6. Fallback Start
    else:
        new_topic = random.choice(list(reminiscence_topics.keys()))
        chat_state["current_topic"] = new_topic
        return f"Let's chat about {new_topic}. {get_new_prompt(new_topic)}"

print("✅ Remi V3 Logic Loaded: No Repeats, understands 'No', and ignores nonsense.")

✅ Remi V3 Logic Loaded: No Repeats, understands 'No', and ignores nonsense.


In [ ]:
print("--- Remi V3 Online ---")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Remi: Bye!")
        break
        
    response = get_smart_response_v3(user_input)
    print(f"Remi: {response}")

--- Remi V3 Online ---


You:  Peter


Remi: Let's chat about family. What is a favorite holiday tradition your family had?


You:  I don't remember


Remi: Understood. Let's move on. What is a professional accomplishment you are most proud of?


...

Remi: I'm not sure I understand. Could you tell me a bit more in a full sentence?


You:  go to hell


Remi: Okay. What was your very first job? Did you enjoy it?


You:  stop asking me questions


Remi: I apologize. I am just a computer program trying to learn. Let's start over. do you have a favorite animal?


The above is important because of context awareness, i.e if the user says I loved my mum, the bot will detect family and ask a family question next. It acts as a safety valve i.e < -0.5 check if the patient gets upset, the bot tries to pivot to a happier subject(music/hobbies)

we are now going to stitch all those pieces together into a running loop. This will be the main chat loop.